In [1]:
import os
import json 

# Load the environment configuration JSON data
json_path = 'env_config.json'
with open(json_path, 'r') as file:
    env_config = json.load(file)

hf_home = env_config['HF_HOME']
# Set the HF_HOME environment variable
os.environ['HF_HOME'] = hf_home
# Set the access token to huggingface hub
access_token = env_config['access_token']
os.environ['HUGGINGFACE_HUB_TOKEN'] = access_token

# Load Model and Tokenizer

In [2]:
import transformers 
print(transformers.__version__)

from transformers import pipeline
import torch

from accelerate import Accelerator
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from transformers import LlamaTokenizerFast


accelerator = Accelerator()
device = accelerator.device

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "meta-llama/Meta-Llama-3-8B"  # non-instruct version

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=access_token,
)

tokenizer = LlamaTokenizerFast.from_pretrained(model_id, token=access_token)

/opt/crc/c/conda/23.5.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.41.0


Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.58s/it]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Load data and define dataloaders

In [3]:
from datasets import load_dataset
from torch.utils.data import DataLoader

imdb = load_dataset("imdb")
train_ds = imdb['train']


def collate_fn(examples):
    max_len = 512
    texts = [example['text'] for example in examples]
    texts = [text if len(text) <= max_len else text[:max_len] for text in texts]
    labels = [example['label'] for example in examples]
    messages_lambda = lambda texts: [
        {"role": "system", "content": "You are a chatbot for sentimate analysis. You can help users with their questions via concise responses of POSITIVE, NEGATIVE OR NEUTURAL with a brief explanation."},
        # {"role": "system", "content": "You are a chatbot for sentimate analysis."},
        {"role": "user", "content": texts},
    ]
    messages = list(map(messages_lambda, texts))

    messages_with_template_applied = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    batch = tokenizer(
                messages_with_template_applied,
                add_special_tokens=False,
                padding=True,
                return_tensors="pt",
                )
    
    # find the template boundaries
    text_lens = [len(tokenizer.encode(text)) for text in texts]
    text_lens_tensor = torch.tensor(text_lens, dtype=torch.long)
    

    def apply_mask(mask_tensor, text_lens_tensor):
        batch_size, seq_len = mask_tensor.shape
        for i in range(batch_size):
            text_len = text_lens_tensor[i].item()
            mask_tensor[i, -text_len-5:-5] = 0
        return 1- mask_tensor

    mask_tensor = apply_mask(torch.ones_like(batch['input_ids']), text_lens_tensor)

    batch['context_mask'] = mask_tensor

    
    return batch


# Define batch size here!
batch_size = 16
train_dataloader = DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)


# Define maskgen model

In [4]:
import torch.nn as nn 
from models import MLP
import torch.nn.functional as F

def similarity_measure(logits, labels, attention_mask):
    """ 
    args:
        logis: torch.Tensor, shape (batch_size, seq_len, vocab_size)
        labels: torch.Tensor, shape (batch_size, seq_len)
        attention_mask: torch.Tensor, shape (batch_size, seq_len) the original input text is masked with 0

    return:
        mean_log_probs: torch.Tensor, shape (batch_size,)
    """
    log_probs = torch.log_softmax(logits, dim=-1)
    # correct_log_probs = log_probs[range(log_probs.shape[0]), labels]

    # Flatten the tensors to simplify indexing
    batch_size, seq_len, vocab_size = logits.size()
    labels = labels.view(-1)  # flatten to (batch_size * seq_len)
    log_probs_flat = log_probs.view(-1, vocab_size)

    # Extract the log probabilities for the correct labels
    correct_log_probs_flat = log_probs_flat[range(batch_size * seq_len), labels]

    # Reshape to (batch_size, seq_len)
    correct_log_probs = correct_log_probs_flat.view(batch_size, seq_len)

    # Mask out the original input texts, only keep the generated tokens
    masked_log_probs = correct_log_probs * attention_mask

    # Calculate the mean log probability for each sequence
    mean_log_probs = masked_log_probs.sum(dim=-1)/ attention_mask.sum(dim=-1)

    return mean_log_probs


class MaskGeneratingModel(nn.Module):
    def __init__(self, hidden_size=4096, mlp_hidden_dim=1024, mlp_bottleneck_dim=768, mlp_num_blocks=2):
        """ 
        hidden_size: int
            The hidden size of the output of the generative model, 4096 for llama3
        """
        super().__init__()

        self.hidden_size = hidden_size

        self.explain_map = MLP(input_dim=hidden_size, 
                               hidden_dim=mlp_hidden_dim, 
                               output_dim=1, 
                               num_blocks=mlp_num_blocks, 
                               bottleneck_dim=mlp_bottleneck_dim) # takes [N, L, hidden_size] outputs [N, L, 1]
    
    def forward(self, pred_features):
        """ 
        pred_features: torch.Tensor of shape [N, L, hidden_size]
        """
        mask_logits = self.explain_map(pred_features) # [N, L, 1]
        return mask_logits.squeeze(-1) # [batch_size, seq_len]
    
    @torch.no_grad()
    def generate_mask(self, mask_logits, context_mask):
        """
        generate mask based on a Bernoulli distribution with the probabilities of the given logits 
        args:
            mask_logits: torch.Tensor, shape (batch_size, seq_len)
            context_mask: torch.Tensor, shape (batch_size, seq_len), the context texts 
                (for example, the chatbot instruction template, not including the user inputs) are masked with 0.
                This mask strategy is to ensure we only focus on the real user inputs.
        return:
            mask: torch.Tensor, shape (batch_size, seq_len), with contexts being all 1s and user inputs being randomly masked.
        """
        mask_probs = torch.sigmoid(mask_logits) # (batch_size, seq_len)
        mask = torch.bernoulli(mask_probs) # (batch_size, seq_len)
        mask = context_mask * mask + (1. - context_mask) # (batch_size, seq_len)
        return mask # this could be used as the attention mask for the generative model
    
    @torch.no_grad()
    def sample_one_batch(self, input_ids, attention_mask, mask_logits, context_mask):
        """ 
        args:
            input_ids: torch.Tensor, shape (batch_size, seq_len)
            attention_mask: torch.Tensor, shape (batch_size, seq_len), the attention_mask generated automatically by tokenizer, usually all 1s
            mask_logits: torch.Tensor, shape (batch_size, prompt_len), the logits for the mask generation
            context_mask: torch.Tensor, shape (batch_size, prompt_len), the context texts 
                (for example, the chatbot instruction template, not including the user inputs) are masked with 0.
        """
        #  get the mask of interest, i.e., for the user inputs, then pad to match the shape of other masks (of the full length)
        seq_len = input_ids.size(1)
        prompt_len = mask_logits.size(1)
        mask = self.generate_mask(mask_logits=mask_logits, context_mask=context_mask) # (batch_size, prompt_len)
        pad_length = seq_len - prompt_len
        padded_mask = F.pad(mask, (0, pad_length), mode='constant', value=1)

        # get the masked attention_mask
        masked_attention_mask = attention_mask * padded_mask

        return input_ids, masked_attention_mask, mask

    
    def compute_similarity(self, logits, labels, attention_mask):
        """ 
        compute the mean log_probs for the generated tokens
        """
        mean_log_probs = similarity_measure(logits, labels, attention_mask)
        return mean_log_probs


# Train model

In [5]:
mask_gen_model = MaskGeneratingModel(hidden_size=4096, mlp_hidden_dim=1024, mlp_bottleneck_dim=768, mlp_num_blocks=2)
mask_gen_model.to(device)


from tqdm import tqdm

# Set pad_token_id if it is not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

pad_token_id = tokenizer.pad_token_id

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

optimizer = torch.optim.Adam(mask_gen_model.parameters(), lr=1e-5)

for epoch in range(10):
    pbar = tqdm(train_dataloader)
    for idx, data in enumerate(pbar):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        context_mask = data['context_mask'].to(device)
        # get generated texts
        gen_outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=128,
            eos_token_id=terminators,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            return_dict_in_generate=True,
            output_scores=True,
        )
        gen_tokens = gen_outputs.sequences
        pad_length = gen_tokens.size(1) - input_ids.size(1)
        # get the attention mask for the generated tokens, and also mask the padding tokens
        gen_attention_mask = F.pad(attention_mask, (0, pad_length), mode='constant', value=1)
        # (gen_tokens != pad_token_id).long() is the tokens mask, 1 for real tokens and 0 for padding tokens
        unpaded_token_mask = (gen_tokens != pad_token_id).long()
        unpaded_token_mask[:, :-pad_length] = 1
        gen_attention_mask = gen_attention_mask * unpaded_token_mask
        # get the response mask, which is the mask for the generated tokens (the user inputs are masked with 0)
        response_mask = gen_attention_mask.clone()
        response_mask[:, :-pad_length] = 0 # TODO: 有问题. 有问题吗？
        # Get the last hidden state for the prompt sequence
        with torch.no_grad():
            prompt_outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True, return_dict=True)
            last_hidden_state = prompt_outputs.hidden_states[-1]
            last_hidden_state = last_hidden_state.float()
        
        mask_logits = mask_gen_model(last_hidden_state)

        
        # mask = mask_gen_model.generate_mask(mask_logits=mask_logits, context_mask=context_mask)
        perturbed_input_ids, masked_attention_mask, user_input_mask = mask_gen_model.sample_one_batch(input_ids=gen_tokens, 
                                                                attention_mask=gen_attention_mask, 
                                                                mask_logits=mask_logits, 
                                                                context_mask=context_mask)
        
        # Get the logits of the perturbed input concatenated with the response
        with torch.no_grad():
            outputs = model(input_ids=perturbed_input_ids.long(), attention_mask=masked_attention_mask, return_dict=True)
            logits = outputs.logits.float()
        
        # Get the ground truth with the original input concatenated with the response
        with torch.no_grad():
            outputs_gt = model(input_ids=gen_tokens, attention_mask=gen_attention_mask, return_dict=True)
            logits_gt = outputs_gt.logits.float()
        
        # Compute the similarity between the logits and the response labels
        labels = gen_tokens.clone()
        labels[:, :-1] = labels[:, 1:]
        labels[:, -1] = pad_token_id
        shift_response_mask = response_mask.clone()
        shift_response_mask[:, :-1] = shift_response_mask[:, 1:]
        shift_response_mask[:, -1] = 0
        sim = mask_gen_model.compute_similarity(logits, gen_tokens, response_mask)
        sim_gt = mask_gen_model.compute_similarity(logits_gt, gen_tokens, response_mask)

        # PPO
        
        reward = torch.exp(sim - sim_gt)
        mask_prob = torch.sigmoid(mask_logits)
        bce_loss = nn.BCELoss(reduction='none')
        reward_loss = reward.unsqueeze(-1) * bce_loss(context_mask * mask_prob, context_mask * user_input_mask) # consider manually implements the BCE loss
        reward_loss = reward_loss.mean()
        # reduce the probability of the masked tokens, weighted by reward.
        mask_loss = (mask_prob * context_mask * (1 - user_input_mask)).sum(dim=-1) / context_mask.sum(dim=-1)
        mask_loss = mask_loss * reward
        mask_loss = mask_loss.mean()

        loss = reward_loss + mask_loss
        pbar.set_description(f"Epoch {epoch+1}, Step {idx+1}: Loss = {loss.item():.4f}, " 
                             f"Reward = {reward.mean().item():.4f},"
                             f"Reward Loss = {reward_loss.item():.4f}, "
                            #  f"Regret Loss = {loss_dict['regret_loss'].item():.4f}, "
                             f"Mask Loss = {mask_loss.item():.4f} "
                            #  f"alt_mask_loss = {loss_dict['alt_mask_loss'].item():.4f} "
                            #  f"mask_mean = {loss_dict['mask_mean'].item():.4f} "
                            #  f"prob_mean = {loss_dict['prob_mean'].item():.4f} "
                             )
        # Train the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # pbar.set_description(f"Epoch {epoch+1}, Step {idx+1}: Loss = {loss.item():.4f}, " 
        #                      f"Reward Loss = {reward_loss.item():.4f}, "
        #                     #  f"Regret Loss = {loss_dict['regret_loss'].item():.4f}, "
        #                      f"Mask Loss = {mask_loss.item():.4f} "
        #                     #  f"alt_mask_loss = {loss_dict['alt_mask_loss'].item():.4f} "
        #                     #  f"mask_mean = {loss_dict['mask_mean'].item():.4f} "
        #                     #  f"prob_mean = {loss_dict['prob_mean'].item():.4f} "
        #                      )
        if idx % 10 == 0:
            print()
        if idx == 20:
            break
    break


        # scheduler.step()
        # pbar.set_description(f"Epoch {epoch+1}, Step {idx+1}: Loss = {loss_dict['loss'].item():.4f}, " 
        #                      f"Reward Loss = {loss_dict['reward_loss'].item():.4f}, "
        #                     #  f"Regret Loss = {loss_dict['regret_loss'].item():.4f}, "
        #                      f"Mask Loss = {loss_dict['mask_loss'].item():.4f} "
        #                     #  f"alt_mask_loss = {loss_dict['alt_mask_loss'].item():.4f} "
        #                      f"mask_mean = {loss_dict['mask_mean'].item():.4f} "
        #                      f"prob_mean = {loss_dict['prob_mean'].item():.4f} "
        #                      )
        # if idx % 10 == 0:
        #     print()
        # if (idx) % 10 == 0:
            
        #     torch.save(mask_gen_model.state_dict(), f'text_mask_gen_model/mask_gen_model_{epoch}_{idx}.pth') 


Epoch 1, Step 1: Loss = 4.5399, Reward = 7.3851,Reward Loss = 2.9794, Mask Loss = 1.5605 :   0%|          | 1/1563 [00:04<1:45:15,  4.04s/it]

Epoch 1, Step 1: Loss = 4.5399, Reward = 7.3851,Reward Loss = 2.9794, Mask Loss = 1.5605 :   0%|          | 1/1563 [00:04<1:52:24,  4.32s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.34 GiB. GPU 0 has a total capacity of 79.14 GiB of which 525.19 MiB is free. Process 3502083 has 51.13 GiB memory in use. Including non-PyTorch memory, this process has 27.48 GiB memory in use. Of the allocated memory 24.71 GiB is allocated by PyTorch, and 2.26 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
labels

NameError: name 'labels' is not defined

In [ ]:
print(sim)
print(sim_gt)

tensor([-18.0588, -16.2596, -16.1931, -18.6757, -16.4191, -17.1269, -17.0800,
        -16.8343, -16.4374, -15.1854, -17.9453, -16.9966, -18.5555, -16.4071,
        -17.2895, -16.3334], device='cuda:0')
tensor([-23.3271, -21.3801, -20.2045, -20.7105, -22.1320, -20.9240, -22.1199,
        -22.5376, -21.7936, -22.5402, -21.7588, -23.8185, -22.8987, -21.4705,
        -21.6159, -21.2276], device='cuda:0')


In [ ]:
idx = 1

print(response_mask[idx])
# print("context_mask\n", context_mask[idx])
# print("user_input_mask\n",user_input_mask[idx].long())
print("masked_attention_mask\n",masked_attention_mask[idx].long())
# print("response_mask\n", response_mask[idx].long())
print("gen_attention_mask\n", gen_attention_mask[idx].long())


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
masked_attention_mask
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 

In [ ]:
perturbed_input_ids[idx] == gen_tokens[idx]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [ ]:
tokenizer.decode(perturbed_input_ids[idx] * masked_attention_mask[idx].long())

NameError: name 'tokenizer' is not defined